In [1]:
# Import
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [2]:
# Load MNIST Datase
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])

train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1000, shuffle=False)

100%|██████████| 9.91M/9.91M [00:00<00:00, 61.4MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 1.71MB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 14.8MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 7.34MB/s]


In [3]:
#  MLP Mode
class MLP(nn.Module):
    def __init__(self, activation_fn=nn.ReLU):
        super(MLP, self).__init__()
        self.model = nn.Sequential(
            nn.Flatten(),
            nn.Linear(28*28, 256),
            activation_fn(),
            nn.Linear(256, 128),
            activation_fn(),
            nn.Linear(128, 10)
        )

    def forward(self, x):
        return self.model(x)

In [5]:
# Training Functio
def train(model, optimizer, criterion, epochs=5):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for data, target in train_loader:
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f"Epoch {epoch+1}, Loss: {total_loss:.4f}")

In [6]:
# Evaluation Function
def evaluate(model):
    model.eval()
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            output = model(data)
            pred = output.argmax(dim=1)
            correct += pred.eq(target).sum().item()
    accuracy = 100. * correct / len(test_loader.dataset)
    print(f"Test Accuracy: {accuracy:.2f}%")

In [7]:
# Experimenting with Activation Function
for act_fn in [nn.ReLU, nn.Tanh, nn.Sigmoid]:
    print(f"\nUsing activation: {act_fn.__name__}")
    model = MLP(activation_fn=act_fn)
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    criterion = nn.CrossEntropyLoss()
    train(model, optimizer, criterion, epochs=5)
    evaluate(model)


Using activation: ReLU
Epoch 1, Loss: 215.6061
Epoch 2, Loss: 86.5530
Epoch 3, Loss: 62.2674
Epoch 4, Loss: 46.7625
Epoch 5, Loss: 38.0076
Test Accuracy: 97.16%

Using activation: Tanh
Epoch 1, Loss: 219.2150
Epoch 2, Loss: 93.5043
Epoch 3, Loss: 67.9490
Epoch 4, Loss: 50.3999
Epoch 5, Loss: 44.9969
Test Accuracy: 97.20%

Using activation: Sigmoid
Epoch 1, Loss: 395.2008
Epoch 2, Loss: 125.1690
Epoch 3, Loss: 78.3173
Epoch 4, Loss: 55.8647
Epoch 5, Loss: 40.4212
Test Accuracy: 97.04%
